In [ ]:
!rm -rf camp_data
!git clone https://github.com/HongJeSeong/camp_data.git

In [ ]:
!ls camp_data

In [ ]:
import pandas as pd
df = pd.read_csv("camp_data/review.txt", names=['sentence', 'label'], sep='\t')
df

In [ ]:
from sklearn.model_selection import train_test_split
x = df['sentence'].values
y = df['label'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=20)

`tokenizer = Tokenizer(num_words=500)`  

데이터를 읽어 들이고
단어 빈도에 따른 사용할 단어 개수의 최대값. 가장 빈번하게 사용되는 num_words개의 단어만 저장  
가장 많이 출현한 단어 Top 500을 선택한다는 의미

`xtrain= tokenizer.texts_to_sequences(x_train)`

Top 500개로 저장한 단어사전에서 매칭되는 단어들을 선택하여 문장 형식으로 변경

![](https://raw.githubusercontent.com/HongJeSeong/camp_data/main/img/wordindex.PNG)



In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(x)
xtrain= tokenizer.texts_to_sequences(x_train)
xtest= tokenizer.texts_to_sequences(x_test) 
vocab_size=len(tokenizer.word_index)+1 # 단어들이 저장된 단어 사전의 크기 + 1(공백단어때문에 +1)
print(vocab_size)

In [ ]:
tokenizer.word_index

In [ ]:
x_train[5], xtrain[5]

# Padding
![padding](https://raw.githubusercontent.com/HongJeSeong/camp_data/main/img/padding.PNG)
(https://raw.githubusercontent.com/HongJeSeong/camp_data/main/img/padding.PNG)

원래의 입력 단어가 SIZE 10을 `넘는다면` ?  
10까지만 넣어지고 나머지 뒤 부분은 잘림

## post, pre

![](https://raw.githubusercontent.com/HongJeSeong/camp_data/main/img/prepost.PNG)

In [ ]:
xtrain=pad_sequences(xtrain,padding='post', maxlen=20)
xtest=pad_sequences(xtest,padding='post', maxlen=20)

In [ ]:
xtrain[1]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout
from tensorflow.keras.layers import Embedding
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=20))
model.add(LSTM(50, dropout=0.5)
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

### Embedding
`Embedding(vocab_size, embedding_vector_length, input_length=20)`
* voca_size(input_dim) : 단어 사전의 크기를, 총 5272개의 단어 종류가 있다는 뜻
* embedding_vector_length(output_dim) : 단어를 인코딩 한 후 나오는 벡터 크기 의미론적 기하공간에 나타낸다는 의미  
즉 임베딩 레이어는 입력되는 단어를 의미론적으로 잘 설계된 공간에 위치시켜 벡터로 수치화 시킨다
* input_length(input_length) : 단어의 수 즉, 문장의 길이를 의미 
임베딩 레이어 다음에 Flatten 레이어가 온다면 반드시 input_lenth를 지정해야 함 
Flatten 레이어인 경우 입력 크기가 알아야 이를 1차원으로 만들어서 Dense 레이어에 전달할 수 있기 때문

![d](https://raw.githubusercontent.com/HongJeSeong/camp_data/main/img/dropout.PNG)
(https://raw.githubusercontent.com/HongJeSeong/camp_data/main/img/dropout.PNG)
학습의 오버피팅을 줄이기 위한 레이어

In [ ]:
history = model.fit(xtrain,y_train, epochs=120, verbose=1,validation_data=(xtest,y_test))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(18, 10))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

In [ ]:
import numpy as np
def pred_result(text,model):
  text= tokenizer.texts_to_sequences(text)
  text=pad_sequences(text,padding='post', maxlen=20)
  pred = model.predict(text)

  print("확률 : ",pred)
  if pred > 0.5:
    print("Positive")
  else:
    print("Negative")


In [ ]:
pred_result(["great"],model)